In [ ]:

import npy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from gensim.models import KeyedVectors

# Load pre-trained word embeddings
word_vectors = KeyedVectors.load_word2vec_format('path/to/pretrained/word2vec.bin', binary=True)

# Load IMDB reviews dataset
df = pd.read_csv("path/to/imdb_reviews.csv")

# Split data into features and labels
X = df['review']
y = df['sentiment']

# Convert labels to binary values (0 or 1)
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

# Tokenize the reviews
tokenizer = keras.preprocessing.text.Tokenizer(n_words=5000)
tokenizer.fit_on_texts(X)

# Convert the text data to sequence
X = tokenizer.texts_to_sequences(X)

# Pad sequences to same length
max_words = 500
X = sequence.pad_sequences(X, maxlen=max_words)

# Replace the word index with pre-trained word embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

# Define model architecture
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 300, weights=[embedding_matrix], input_length=max_words, trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Use early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

# Train the model
batch_size = 64
n_epochs = 10
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=n_epochs, callbacks=[early_stopping])

# Evaluate the model on test data
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

# Define the path to save the model
model_path = '/content/drive/My Drive/my_model.h5'

# Save the model
model.save(model_path)

# Load the model
loaded_model = keras.models.load_model(model_path)

# Define a function to predict sentiment
def predict_sentiment(review):
    # Tokenize and pad the review
    review_seq = tokenizer.texts_to_sequences([review])
    review_seq = sequence.pad_sequences(review_seq, maxlen=max_words)
    # Predict the sentiment using the loaded model
    prediction = loaded_model.predict(review_seq)[0
```